# Testing different SA methods 1/5
## nltk's naive bayes
This code was partly taken from [here](http://www.laurentluce.com/posts/twitter-sentiment-analysis-using-python-and-nltk/)
TODO also attribute in thesis

In [15]:
import nltk
import csv
import pickle
import random

# Ugly hackery, but necessary: stackoverflow.com/questions/4383571/importing-files-from-different-folder
import sys
sys.path.append('../../../')
    
from src.streaming import spark_functions

In [16]:
preprocess = spark_functions.preprocessor()

# This is specific to the dataset
negative_label = 'negative'
neutral_label = 'neutral'
positive_label = 'positive'
irrelevant_label = 'irrelevant'

with open('./../../../data/interim/sanders_hydrated.csv') as csv_file:
    iterator = csv.reader(csv_file, delimiter=',')
    # Load the parts we need and preprocess the text
    tweets = [(preprocess(text), sentiment) for (topic, sentiment, id, text) in iterator]
    # The tweets are ordered, so shuffle for the training/test split
    random.shuffle(tweets)

In [17]:
tweets[0]

('Microsoft announces official SQL Server driver for Linux SqlServer',
 'neutral')

In [18]:
"There are %d tweets in this dataset" % len(tweets)

'There are 4347 tweets in this dataset'

In [8]:
# Split between training and test set
# 90% training, 10% test
threshold = int(0.9 * len(tweets))
train_tweets = tweets[:threshold]
test_tweets = tweets[threshold:]

In [10]:
# Use the dictionary we already created for the topic model
# TODO create the dictionary with a separate script and also prune it and use it here and for wordcount

In [11]:
# TODO there might be a function like this in nltk?
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in dictionary.id2token:
        features['contains(%s)' % word] = (word in dictionary)
    return features

In [12]:
training_set = nltk.classify.apply_features(extract_features, train_tweets)

In [13]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [14]:
tweet = 'Larry is not my friend'
print(classifier.classify(extract_features(tweet.split())))

neutral


In [17]:
labeled_correctly = 0
for tweet in test_tweets:
    if classifier.classify(extract_features(tweet[0])) == tweet[1]:
        labeled_correctly+=1
print("%d/%d = %.2d percent" % (labeled_correctly, len(test_tweets), labeled_correctly/len(test_tweets)*100))

693/845 = 82 percent


In [15]:
# Save the classifier if it's any good
save_classifier = open("./../../models/naive_bayes/nltk_naive_bayes.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()